# Introducción al manejo de datos geográficos

## Clase 2: manejo de APIs geográficas y estrategias de visualización

A lo largo del notebook anterior vimos algunas cuestiones generales del manejo de datos espaciales. Los tipos de geometrías y su implementación en `shapely`, los métodos propios a cada una de ellas, manipulación de geodataframes con `geopandas` y, también, algún avance de cómo visualizar sobre un mapa las transformaciones que ibamos efectuando sobre nuestros datos.

Ahora, intentaremos hacer un repaso de las librerías más utilizadas para el ploteo de información geográfica. Estas herramientas son de gran utilidad ya que, combinándolas con los recursos que venimos revisado, amplían las posibilidades que brinda un SIG de escritorio clásico como QGIS o ARCGIS.

Para usarlas, construiremos antes el problema que queremos trabajar espacialmente. Para ello, partiremos de una circunstancia bastante frecuente en el mundo de los datos geográficos: la ausencia de coordenadas o de algún otro atributo que permita su la rápida visualización sobre un layer. Este problema lo vamos a resolver revisando algunas APIs de normalización y enriquecimiento de unidades geográficas. 

A partir de un caso concreto, introduciremos dos APIs diferentes. Y con cada una de ellas, daremos respuesta al mismo problema. Reconstruiremos primero la dimensión espacial de nuestros datos y compararemos resultados apelando a distintas librerías de visualización. Comencemos!

### Preparación de los datos

Una situación bastante común cuando trabajamos con datos que tienen una representación espacial, es que esta no esté disponible como columna o Serie de un DataFrame de pandas. En otras palabras, que no dispongamos de la latitud y la longitud de los datos que buscamos representar espacialmente. Pongamos un ejemplo a partir de un caso de uso real y veamos cómo solucionarlo. 

Supongamos que nuevamente necesitamos trabajar con edificios. Pero en esta oportunidad, no con desarrollos inmobiliarios sino con edificios que han sido catalogados como patrimonio histórico de la ciudad. 

A continuación veremos un [listado de fachadas](https://data.buenosaires.gob.ar/dataset/fachadas) con certificado de conservación. Este dataframe lo bajamos del portal de datos del Gobierno de la Ciudad de Buenos Aires, comúnmente conocido como [data buenos aires](https://data.buenosaires.gob.ar/dataset) - un sitio de consulta de datasets de bastante utilidad.

![FACHADAS](imagenes/fachadas.png)

In [ ]:
# Damos un primer vistazo a nuestro DataFrame
import pandas as pd

In [ ]:
pd.read_csv('data/fachadas.csv').head()

Como se puede apreciar, este listado cuenta con la dirección física de cada una de las fachadas. Pero si quisiéramos representarlas en un mapa necesitaríamos de una serie de atributos que no están provistos. 

## Servicio de Normalización de Datos Geográficos (MM)

En el notebook anterior vimos qué es un geocodificador y cómo se puede obtener las coordenadas geográficas de un punto específico a partir de la API de Google. Ahora vamos a ampliar un poco el espectro e introduciremos otra herramienta del estilo que desarrolló el Gobierno Nacional. Esta es la [API del Servicio de Normalización de Datos Geográficos de Argentina](https://datosgobar.github.io/georef-ar-api/), una herramienta muy útil para normalizar entidades territoriales, enriquecerlas u obtener información como sus coordenadas.

Esta herramienta nos será de mucha utilidad para el problema que necesitamos abordar: `obtener la latitud y longitud de un punto a partir de un string de direcciones físicas`.

In [ ]:
# Primero instanciemos nuestro dataframe y construyamos nuestro string de direcciones
fachadas = pd.read_csv('data/fachadas.csv')

In [ ]:
# Como la altura es un valor numérico,
fachadas.calle_altura.dtype

In [ ]:
# vamos a reemplazar los NaN por cero
fachadas.calle_altura.fillna(0, inplace=True)

In [ ]:
# y ver cuántos son. Como es bajo el número vamos a suponer que los faltantes corresponden a la altura 0.
len(fachadas.loc[fachadas.calle_altura == 0])

In [ ]:
# Y ahora convertimos las alturas en integer,
fachadas.calle_altura = fachadas.calle_altura.map(int)

In [ ]:
# para luego guardarlas como string.
fachadas.calle_altura = fachadas.calle_altura.astype(str)

In [ ]:
# Armamos un único string
fachadas['direccion'] = fachadas['calle_nombre'] + ' ' + fachadas['calle_altura']

Ahora que ya tenemos nuestro string listo, veamos un primer ejemplo para enriquecer nuestras entidades geogŕaficas revisando algunos de los [ejemplos en python](https://datosgobar.github.io/georef-ar-api/python-usage/).

Lo que vamos a hacer es tomar el caso de normalización de varias entidades que se lleva a cabo para consultas provinciales. Por medio de la función `similar_bulk` se construirá un diccionario con el listado de direcciones. Se utilizará el parámetro `endpoint`, con el nombre de lo que será la llave con la que se accederá a una lista de diccionarios. Cada uno de esos diccionarios contará con la key `dirección` que dará acceso al `str` que construimos más arriba con el nombre de la dirección. Es decir, la lista de direcciones que pasamos como segundo parámetro. 

También vamos a agregar en el cuerpo de la función una información adicional: el `departamento` al que pertenece nuestra entidad. Así es como se guarda la data que será consultada en el endpoint a través del request. Las keys dirección y departamento son las que darán acceso a la base de datos de donde se consumirá la información adicional que nos traeremos de vuelta. 

Para esto nos fijamos cómo se construye la API BASE para direcciones en los [ejemplos de uso](https://datosgobar.github.io/georef-ar-api/quick-start/). Así, podemos ver que el formato de la consulta es el siguiente: `https://apis.datos.gob.ar/georef/api/direcciones?departamento=merlo&direccion=Florida al 2950`. En nuestro caso, como estamos en la Ciudad de Buenos Aires nuestro departamento serán las Comunas. Al tener 15 posibilidades, podría ser que debiéramos haber completado con el `str` de las `15 Comunas`. Para lo cual, deberíamos haber incorporado un parámetro adicional en la función `get_similar_bulk`. Algo que hiciera masomenos lo siguiente:

```
def get_similar_bulk(endpoint, nombres, departamento):
    (...)
    return parsed_results
```

Pero eso hubiese llevado a la tarea de introducir la consulta en un loop que iterara sobre el rango 1 a 15 y testeara las quince posibilidades diferentes para cada dirección - dado que por la información con la que cuenta nuestra base, tampoco sabemos a qué comuna pertenece cada una de ellas-. Algo verdaderamente costoso en consumo de memoria. Pero por suerte la API resuelve muy bien este caso de uso. Con sólo utilizar la palabra `Comuna` para el parámetro `departamento` que se consumirá en el endpoint será suficiente para traer la consulta sin error.  

In [ ]:
# Importemos los módulos necesarios para el uso de la API
import requests

In [ ]:
# Construimos un string con la url base que vamos a usar para construir las consultas
API_BASE_URL = "https://apis.datos.gob.ar/georef/api/"

In [ ]:
# Creamos la función
def get_similar_bulk(endpoint, nombres):
    '''
    Normaliza una lista de nombres de alguna de 
    las entidades geográficas.
    ...
    Argumentos:
        endpoint(str): String de texto.
        nombres (iter): Objeto iterable (e.g: list, Series)
    Devuelve:
        list: diccionarios con el resultado de la consulta 
    '''

    # realiza consulta a la API
    data = {
        endpoint: [
            {"direccion": nombre,
             "departamento":'Comuna',
             "max":5} for nombre in nombres
    ]}
    url = API_BASE_URL + endpoint
    results = requests.post(
        url, json=data, headers={"Content-Type": "application/json"}
    ).json()
    
    try:
        parsed_results = [
            single_result[endpoint][0] if single_result[endpoint] else {'Sin dato'}
            for single_result in results["resultados"]
        ]
        print('Consulta realizada: %r resultados obtenidos' % len(parsed_results))

    except:
        print('Excediste el limite de consultas')
        parsed_results = {'Sin dato'}

    return parsed_results

Si comparamos la función con la del ejemplo provisto en la web de la API se puede identificar que agregamos un `try`/`except` al final. Antes de usar este tipo de servicios, es recomendable leer las [condiciones de uso](https://datosgobar.github.io/georef-ar-api/terms/). En el caso de esta API, las [consultas por lote](https://datosgobar.github.io/georef-ar-api/bulk/) tienen límites máximos. Veamos qué pasa si no los respetamos...  

In [ ]:
# Este es el total de direcciones que deberíamos consultar
len(fachadas.direccion)

In [ ]:
nombres = fachadas.direccion

In [ ]:
# Podemos ver que en algún momento de la consulta, la API dejó de responder y pasamos al bloque except.
direcciones = get_similar_bulk("direcciones", nombres)

Esto es porque la cantidad de consultas en una misma petición no debe superar las `1000`. Nosotros definimos 5 como `max`, con lo cual si ahora vamos por 999 no deberían tener ningún problema en hacer la consulta.

In [ ]:
import numpy as np

In [ ]:
def consultas(totales):
    '''
    Aplica get_similar_bulk sobre
    la serie de direcciones de un df
    ...
    Argumentos:
        totales (int): límite superior del intervalo.
    Devuelve:
        list: diccionarios con el resultado de cada consulta 
    '''
    intervalos = np.arange(0, totales, 999).tolist()
    intervalos.append(totales)
    
    listado = []
    for i in range(len(intervalos)):
        try:
            print("Iterando de %r a %r casos" % (intervalos[i], intervalos[i+1]))
            I = intervalos[i]
            F = intervalos[i+1]
        except:
            print("Consulta terminada")
            break

        nombres = fachadas.direccion.iloc[I:F]
        nombres.fillna('Sin dato', inplace=True)
        direcciones = get_similar_bulk("direcciones", nombres)
        listado.extend(direcciones)
        
    return listado

In [ ]:
# Aplicamos get_similar_bulk sobre todo el listado de direcciones de nuestro df
para_enriquecer = consultas(len(fachadas))

In [ ]:
# Tenemos la misma cantidad de casos en nuestra consulta
len(para_enriquecer)

In [ ]:
# y en nuestro dataframe original
len(fachadas)

In [ ]:
# Y estas son las keys que tenemos en cada consulta. Ponemos la primera sólo para verlas.
para_enriquecer[0].keys()

In [ ]:
def enriquece(consultas, atributo):
    '''
    Accede al item de cada key de 
    obtenido en la conuslta.
    ...
    Argumentos:
        consultas (list): lista de diccionarios.
        atributo (str): nombre de la key
    Devuelve:
        list: string con el atributo consultado 
    '''
    resultado = []
    for i in range(len(consultas)):
        if type(consultas[i]) is dict:
            if atributo in consultas[i].keys():
                resultado.append(consultas[i][atributo])
        else:
            resultado.append('Sin resultado')
    return resultado

In [ ]:
# Nos traemos la nomenclatura y vemos que seguimos teniendo la misma cantidad de casos. Es un buen signo!
len(enriquece(para_enriquecer, 'nomenclatura'))

In [ ]:
# Ahora sumamos esta info a nuestro dataframe
fachadas['nomenclatura'] = enriquece(para_enriquecer, 'nomenclatura')

In [ ]:
fachadas.head()

Ahora tenemos una columna de nomenclatura con toda la dirección completa. Cómo sería enriquecer nuestro dataframe con el resto de los atributos? Por ejemplo, con la información del departamento...

In [ ]:
# Como se puede ver, algunos casos son listas de diccionarios donde cada key es un potencial atributo...
pd.Series(enriquece(para_enriquecer, 'departamento')).head(3)

Para estos casos, vamos a construir una funcion que nos permita ir recolectando los atributos de interés. Por ejemplo...

In [ ]:
def devuelve_subatributo(atributo, subatributo):
    '''
    Accede a los items de las listadas almacenadas
    dentro del diccionario.
    ...
    Argumentos:
        atributo(str): string de texto con el nombre de la key.
        subatributo (list): lista de string con el nombre de
                            de los subatributos.
    Devuelve:
        item: elemento de cada key ordenada en la lista. 
    '''
    atributos = enriquece(para_enriquecer, atributo)
    return [atributos[i][subatributo] if type(atributos[i]) 
            is dict else 'Sin dato' for i in range(len(atributos))]

In [ ]:
# Construimos los parametros de la función
targets = {'departamento':['id','nombre'],
           'ubicacion':['lat','lon'],
           'localidad_censal':['id','nombre']}

In [ ]:
# y la aplcamos en un for loop.
for k,v in targets.items():
    print(k, v[0], v[1])
    fachadas[v[0]],fachadas[v[1]] = devuelve_subatributo(k,v[0]), devuelve_subatributo(k,v[1])

**Ahora, qué problema ven con esto? Se guardan las columnas como nosotros queremos o hay algún efecto no deseado?** 

**Podrían identificar cuál es el problema?** 

In [ ]:
# Solución
for k,v in targets.items():
    print(k, v[0], v[1])
    if k == 'departamento':
        fachadas[v[0]+'dto'],fachadas[v[1]+'dto'] = devuelve_subatributo(k,v[0]), devuelve_subatributo(k,v[1])
    else:
        fachadas[v[0]],fachadas[v[1]] = devuelve_subatributo(k,v[0]), devuelve_subatributo(k,v[1])     

Efetivamente, como dos de nuestros atributos tenían el mismo nombre, el último pisaba el primero y por eso sólo veíamos las columnas de id y nombre del último caso iterado. Veamos ahora nuestro dataframe enriquecido...

In [ ]:
fachadas.head(2)

In [ ]:
# Exportemoslo para no tener que correr todo de nuevo
#fachadas.to_csv('fachadas_sndg.csv', index=False)

## Normalizador AMBA (USIG-GCBA)

Otro servicio similar es el [normalizador de direcciones](https://pypi.org/project/usig-normalizador-amba/) del Gobierno de la Ciudad de Buenos Aires. Antes de comenzar a utilizarlo, vamos a hacer algunas aclaraciones. Revisando el [release history](https://pypi.org/project/usig-normalizador-amba/#history), constatamos que la [versión más reciente](https://github.com/usig/normalizador-amba) debería estar en `1.3.0`. Sin embargo, una versión más reciente (`2.1.2`) se encuentra disponible en [sitios oficiales](http://servicios.usig.buenosaires.gob.ar/normalizar). Aparentemente esta aún no se encuentra disponible en `pip`. Por lo tanto, iremos recorriendo algunos ejemplos de la última release de Git y revisando el instructivo disponible en el último hipervínculo compartido más arriba. Prosigamos...

In [ ]:
# Instalemos desde pip si es que aún no lo hiciste
#!pip install usig-normalizador-amba

In [ ]:
# Importamos el normalizador
from usig_normalizador_amba import NormalizadorAMBA

Algo interesante de esta API (que no vimos en la anterior) es que permite normalizar una dirección parseando strings de texto y eligiendo el distrito objetivo (esto siempre dentro del Área Metropolitana de Buenos Aires). En la función que armamos a continuación vemos dos formas diferentes de instanciar el normalizador. Introduciendo un `Boolean` parameter vamos a elegir cuál de los dos usar. Ambos devuelven la misma información pero nos sirven para resolver situaciones diferentes. Si nuestro input ya se encuentra formateado como dirección, es decir, no se encuentra con formatos del estilo `calle 1, esquina calle 2` - por mencionar sólo un ejemplo - utilizaremos el normalizador simplemente. Mientras que en el segundo caso lo instanciaremos como `buscarDireccion`.  

In [ ]:
def normaliza(direccion, parsea_texto):
    '''
    Normaliza una dirección geográficas.
    ...
    Argumentos:
        dirección (str): String de texto.
        parsea_texto (bool): Buscar dirección en texto crudo.
    Devuelve:
        dict: nombre, altura y coords de la dirección. 
    '''
    nd = NormalizadorAMBA(include_list=['caba'])
    string = u'{}'.format(direccion)
    
    try:
        if parsea_texto:
            res = nd.buscarDireccion(string)
            for r in res:
                n = res[0][0]['direcciones'][0].calle.nombre
                a = res[0][0]['direcciones'][0].altura
                c = res[0][0]['direcciones'][0].coordenadas
                
            return {'nombre': n, 
                    'altura': a,
                    'coord': c}
        
        else:
            res = nd.normalizar(string)
        
            for r in res:
                n = r.calle.nombre
                a = r.altura
                c = r.coordenadas

            return {'nombre': n, 
                    'altura': a,
                    'coord': c}
    
    except Exception as e:
        print('error')

In [ ]:
# Apliquemos esta función sobre las dos primeras direcciones de nuestro dataframe
fachadas.direccion[0:2]

In [ ]:
# Como las direcciones están en una serie de pandas podemos usar la función anónima lambda
fachadas.direccion[0:2].apply(lambda x: normaliza(x, False))

In [ ]:
# Probemos cómo hubiese sido si nuestras direcciones se encontraban en un formato más crudo
pd.Series(['Julián Álvarez al 2531', 'emilio lamarca 1014 (y galicia)']).apply(lambda x: normaliza(x,'Texto'))

Aparentemente, en nuestro dataframe las direcciones ya se encontraban normalizadas. Esto lo podemos constatar viendo cómo se normalizan los casos en formato más crudo. Ahora bien, parece haber un problema. Y este es que en ninguna de las dos oportunidades nos duevle coordenadas:

In [ ]:
#1
print(fachadas.direccion[0:2].apply(lambda x: normaliza(x, False))[0]['coord'])

In [ ]:
#2
print(pd.Series(['Julián Álvarez al 2531', 
                 'emilio lamarca 1014 (y galicia)']).apply(lambda x: normaliza(x,'Texto'))[0]['coord'])

Antes de continuar, evaluemos un poco la performance del normalizador. Qué tiempo nos llevaría correrlo sobre toda nuestra serie de direcciones?

In [ ]:
import time
# calculamos tiempo de ejecución
inicio = time.time()
partes = fachadas.direccion[0:1000].apply(lambda x: normaliza(x, 'Texto'))
fin = time.time()

In [ ]:
print('Tiempo de consulta para 1000 casos: %r segundos' % (fin-inicio))

In [ ]:
# Eso quiere decir que haberlo aplicado a toda nuestra serie hubiese llevado...
tiempo_total = (((fin - inicio)*26000)/1000)/60
print('Tiempo de consulta para la totalidad de casos: %r minutos' % tiempo_total)

### Modularizando el normalizador...

Dado que hasta ahora no hemos conseguido las coordenadas geográficas con el normalizador, vamos a combinar algunos pasos que encapsularemos en una función única. Como se ejemplifica en esta [consulta](http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion=juli%C3%A1n%20alvarez%202351,%20caba), la url `http://servicios.usig.buenosaires.gob.ar/normalizar` puede ser completada con una `/direccion=(...)` que devuelve la metadata de la misma. 

Lo que vamos a hacer a continuación es disponer de la librería `urllib` para para traernos ese resultado. Para ello, mostraremos dos caminos alternativos pero con el mismo punto de llegada. Uno largo y otro corto. En el primero veremos cómo trabajar con recursos en su mayoría nativos de python. En el segundo, apelando a métodos específicamente diseñados para lidiar con la situación que debemos sortear.

### EL CAMINO LARGO

El problema que tenemos que resolver es básicamente la adaptación del formato de nuestros strings de direcciones. Como necesitamos que estos nombres se ubiquen en el contexto de una url, es importante que los mismos sean legibles en código [ASCII](https://www.w3schools.com/charsets/ref_html_ascii.asp) para que la consulta se efectúe correctamente

<img align="left" width="500" height="125" src="imagenes/camino_largo.jpeg" style="float: left; padding: 0 15px">
Lo que especificamente necesitamos hacer es enviar un request a una página web utilizando un URL. Las URLs pueden enviarse a Internet solamente con un formato ASCII válido. Dado que nuestros string de direcciones van a contener caracteres que están por fuera de este encoding, debemos garantizar que los caracteres no seguros sean transformados en el contexto mismo de la URL. 

Casos como espacios entre palabras o tildes deberán ser reemplazados por su equivalente en ASCII. Tal es así que en esta seccion mostraremos cómo construir la dirección que debemos incluir en la consulta de una manera más pythonesca, apelando a métodos que son aplicables al string de texto que iremos modificando para darle un encoding compatible a la url. Realizaremos algunas [transormaciones](https://www.w3schools.com/tags/ref_urlencode.ASP) que harán que la dirección que enviemos sea legible en el request.

Comencemos entonces por crear el grupo de funciones con las que modificaremos nuestra serie de direcciones. Lo que haremos por medio de estas es, en primer lugar, separar el nombre de la dirección de su altura y completar los espacios entre strings con el código ASCII que los reemplaza.

In [ ]:
def separa_partes(string):
    '''
    Divide caracteres alfa numéricos dentro
    de un string.
    ...
    Argumentos:
        string(str): String de texto.
    Devuelve:
        dict: calle y altura. 
    '''
    calle, altura = '', ''

    for i in string:
        if i.isdigit():
            altura+=str(i)
        else:
            calle+=str(i)
            
    return {'nombre': calle,
            'altura': altura}        

In [ ]:
# Vemos el resultado de nuestra función
separa_partes(fachadas.direccion[1])

In [ ]:
def unifica_direccion(calle, altura):
    '''
    Unifica caracteres alfanuméricos
    reemplazando espacios por %20
    ...
    Argumentos:
        calle(str): string de letras.
        altura(str): string de numeros
    Devuelve:
        str: direccion unificada. 
    '''
    nombre_url = []
    for i in calle:
        if i != ' ':
            nombre_url.append(i)
        else:
            nombre_url.append('%20')
    nombre_url.extend('%20')
    
    nombre_completo = ''.join([''+i for i in nombre_url])
    
    return nombre_completo+str(altura)

In [ ]:
# Instanciamos el resultado de nuestra función, 
resultado = unifica_direccion(separa_partes(fachadas.direccion[0])['nombre'],
                              separa_partes(fachadas.direccion[0])['altura'])

In [ ]:
# y vemos qué nos devuelve. 
resultado

Como se puede apreciar en el ejemplo de [consulta](http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion=juli%C3%A1n%20alvarez%202351,%20caba) que compartimos previamente, la dirección se encuentra codificada en un formato legible para la consulta que haremos a la url. Lo que hicimos acá es reemplazar los espacios por un `%20` para que nuestro string se acomode al encoding de ASCII. Ahora, vamos a proceder a hacer el request en sí.

In [ ]:
# Importamos el módulo request de urllib,
import urllib.request

In [ ]:
# Instanciamos el string de la consulta sobre el que iremos cambiando el valor de la dirección
url = 'http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion={},%20caba'.format(resultado)

In [ ]:
# Por ejemplo, con lo que guardamos en 'resultado'
contents = urllib.request.urlopen(url).read()

In [ ]:
# esto es lo que nos devuelve
contents

In [ ]:
# Ahora convertimos el btype bytes en un diccionario para recolectar nuestros resultados
import json

In [ ]:
# para lo que cargamos nuestro resultado como json
coords = json.loads(contents.decode('utf-8'))

In [ ]:
coords['direccionesNormalizadas'][0]['coordenadas']

Ahora sí, modularizamos estos pasos en una funcion que nos devuelva las coordenadas. Lo que primero hará es adaptar el string de entrada al formato que usaremos en el url. Luego, lo introducirá en la consulta y, si esta fuese errónea pasará nuevamente el string por el normalizador. Este buscará una dirección en el texto que le pasamos para devolver otra normalizada que iniciará nuevamente el proceso de formateo para reintentar la consulta. 

In [ ]:
def devuelve_coordenadas(direccion):
    '''
    Procesa una dirección y realiza una consulta 
    web al normalizador USIG.
    ...
    Argumentos:
        direccion(str): string de letras y numeros.
    Devuelve:
        dict: epsg, longitud y latitud de la direccion. 
    '''
    
    try:
        # Contruye formato compatible con urlopen
        tildes = {'á': 'a',
                  'é': 'e',
                  'í': 'i', 
                  'ó': 'o',
                  'ú': 'u'}

        for k in tildes.keys():
            if k in direccion:
                direccion = direccion.replace(k, tildes[k])
            else:
                pass

        formato_web = unifica_direccion(separa_partes(direccion)['nombre'],
                                        separa_partes(direccion)['altura'])

        url = 'http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion={},%20caba'
        consulta = url.format(formato_web)

        # Consulta el servicio de georreferencimiento y carga los resultados
        print('Direccion ingresada: %s' % direccion)
        resultado = urllib.request.urlopen(consulta).read()
        carga_resultado = json.loads(resultado.decode('utf-8'))
        print('Direccion conultada: %s' % direccion)


        # Si la consulta falla, volvemos a pasar una direccion normalizada
        if 'errorMessage' in carga_resultado.keys():
            # Normalizamos la direccion para pasar un string legible
            normalizar = normaliza(direccion, 'Texto')
            direccion_normalizada = normalizar['nombre']+' '+str(normalizar['altura'])
            print('Consultando direccion: %s' % direccion_normalizada)
            nvo_formato_web = unifica_direccion(separa_partes(direccion_normalizada)['nombre'],
                                                separa_partes(direccion_normalizada)['altura'])

            nva_consulta = url.format(nvo_formato_web)
            nvo_resultado = urllib.request.urlopen(nva_consulta).read()
            nva_carga_resultado =json.loads(nvo_resultado.decode('utf-8'))
            info = nva_carga_resultado['direccionesNormalizadas'][0]['coordenadas']
        else:
            info = carga_resultado['direccionesNormalizadas'][0]['coordenadas']

        return info
    except:
        return 'Calle inexistente'

In [ ]:
# Recordamos que fachadas.direccion[0] es
fachadas.direccion[0]

In [ ]:
# Probamos nuestra función
devuelve_coordenadas('ALVAREZ, JULIAN 2531')

In [ ]:
# Y ahora vemos qué hubiese devuelto si la dirección estaba escrita de otra manera
devuelve_coordenadas('jUlián álVarez 2531 (esquina arenales)')

In [ ]:
# Vemos cuanto tarda en consultar mil casos
inicio = time.time()
coordenadas = fachadas.direccion[0:1000].map(devuelve_coordenadas)
fin = time.time()

In [ ]:
tiempo_total = (((fin - inicio)*26000)/1000)/60
print('Tiempo de consulta para la totalidad de casos: %r minutos' % tiempo_total)

### EL CAMINO CORTO 

<img align="right" width="525" height="125" src="imagenes/camino_corto.jpeg" style="float: right; padding: 0 15px">

En lugar de adaptar manualmente los caracteres que están por fuera del encoding de la url, ahora vamos a apelar a una herramienta pensada específicamente para codificar una url. Esta es el módulo [parse de urllib](https://www.urlencoder.io/python/). Con el método `quote()` podemos acomodar cada uno de los caracteres de nuestras direcciones sin tener que estar atentos a cada uno de ellos específicamente. 

Esto nos permitirá ahorrar bastantes líneas de código y acortar considerablemente el cuerpo de nuestra función `devuelve_coordenadas`. Ya no buscaremos espacios o tildes, por ejemplo. Simplemente, pasaremos nuestro string por este método y obtendremos un resultado legible en ASCII encoding.  


In [ ]:
# Importamos parse
import urllib.parse

# y utilizamos el método quote para obtener un string en formato ascii
urllib.parse.quote(fachadas.direccion[0]) 

In [ ]:
# Veamos cómo el método quote nos permite acortar la función
def devuelve_coordenadas(direccion):
    '''
    Procesa una dirección y realiza una consulta 
    web al normalizador USIG.
    ...
    Argumentos:
        direccion(str): string de letras y numeros.
    Devuelve:
        dict: epsg, longitud y latitud de la direccion. 
    '''
    
    try:
        # Contruye formato compatible con urlopen
        url = 'http://servicios.usig.buenosaires.gob.ar/normalizar/?direccion={},%20caba'
        direccion_ascii = urllib.parse.quote(direccion)
        consulta = url.format(direccion_ascii)

        # Consulta el servicio de georreferencimiento y carga los resultados
        print('Direccion ingresada: %s' % direccion)
        resultado = urllib.request.urlopen(consulta).read()
        carga_resultado = json.loads(resultado.decode('utf-8'))
        print('Direccion consultada: %s' % direccion)
        info = carga_resultado['direccionesNormalizadas'][0]['coordenadas']
        return info
    except:
        return 'Calle sin resultado'

In [ ]:
# Ejecutemos la función y veamos cuánto tiempo lleva
inicio = time.time()
coordenadas = fachadas.direccion[0:1000].map(devuelve_coordenadas)
fin = time.time()

In [ ]:
tiempo_total = (((fin - inicio)*26000)/1000)/60
print('Tiempo de consulta para la totalidad de casos: %r minutos' % tiempo_total)

Evaluando uno contra otro, no es que el camino corto nos permitirá ejecutar nuestra función de una manera más rápida. Las dos tardan aproximadamente lo mismo. Algo que era esperable, porque ambas deben hacer una consulta (lo único en lo que difieren es la manera en la que codifican el string). Cuestión que no es menor, ya que apelando al método `quote` no sólo conseguimos que nuestra función ahorre líneas de código. También nos asegura no estar sujeto a posibles excepciones (caracteres que no conocemos en nuestra serie de direcciones dada su extensión). Este método estandariza lo que haga falta de manera general sin tener que estar atento caracter por caracter, lo que indudablemente hace que nuestra función sea más consistente. 

In [ ]:
# mapeamos nustra función a toda la serie de direcciones
coordenadas = fachadas.direccion.map(devuelve_coordenadas)

In [ ]:
# Nos devolvió al menos algún valor para todos nuestros casos. La extensión es igual al de la serie de direcciones.
len(coordenadas)

In [ ]:
# Por cada row nos devuelve un diccionario que debemos transformar en columnas.
pd.Series(coordenadas)[0:3]

In [ ]:
# Hacemos una copia de nuestro dataframe de fachadas
fachadas_ = fachadas.copy()

In [ ]:
# Accedemos a la key srid de cada item de nuestra serie de direcciones con una lista por comprension
epsg = [coordenadas[i]['srid'] if coordenadas[i] != 'Calle sin resultado' else 
        'Sin dato' for i in range(len(coordenadas))]

In [ ]:
# El resultado es el que esperabamos, tenemos la misma cantidad de casos.
len(epsg)

In [ ]:
# Hacemos lo mismo con la longitud
lon = [coordenadas[i]['x'] if coordenadas[i] != 'Calle sin resultado' else 
        'Sin dato' for i in range(len(coordenadas))]

In [ ]:
len(lon)

In [ ]:
# y la latitud.
lat = [coordenadas[i]['y'] if coordenadas[i] != 'Calle sin resultado' else 
        'Sin dato' for i in range(len(coordenadas))]

In [ ]:
# Y guardamos todo como series de nuestro df de fachadas
fachadas_['lat'], fachadas_['lon'], fachadas_['epsg'] = lat, lon, epsg

In [ ]:
fachadas_.head()

In [ ]:
# Exportemoslo para no tener que correr todo de nuevo
#fachadas_.to_csv('data/fachadas_usig.csv', index=False)

In [ ]:
pd.read_csv('data/fachadas_usig.csv')

# COMPARACION DE RESULTADOS

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import pyproj

In [ ]:
fachadas_sndg = pd.read_csv('data/fachadas_sndg.csv')

In [ ]:
fachadas_sndg.head(2)

In [ ]:
fachadas_usig = pd.read_csv('data/fachadas_usig.csv')

In [ ]:
fachadas_usig.head(2)

In [ ]:
def construye_coords(x,y):
    try:
        geometria = Point(float(x), float(y)) 
        return geometria
    except:
        return 'sin coordenadas'

In [ ]:
def construye_gdf(df,proj):
    
    geom = df.apply(lambda x: construye_coords(x.lon, x.lat), axis=1)
    df['geometry'] = geom
    localizables = df.loc[df['geometry']!='sin coordenadas']
    
    if type(proj) == str:
        crs = pyproj.CRS.from_user_input(proj)
        
    else:
        crs = pyproj.CRS(proj)
    
    gdf = gpd.GeoDataFrame(localizables, crs=crs, geometry=localizables.geometry)
    print('Devolviendo geodataframe con {} casos perdidos'.format(len(df)-len(gdf)))
    return gdf

In [ ]:
wkt = """PROJCS["GKBA",
        GEOGCS["International 1909 (Hayford)",
            DATUM["CAI",
                SPHEROID["intl",6378388,297]],
            PRIMEM["Greenwich",0],
            UNIT["degree",0.0174532925199433]],
        PROJECTION["Transverse_Mercator"],
        PARAMETER["latitude_of_origin",-34.6297166],
        PARAMETER["central_meridian",-58.4627],
        PARAMETER["scale_factor",0.999998],
        PARAMETER["false_easting",100000],
        PARAMETER["false_northing",100000],
        UNIT["Meter",1]]"""

In [ ]:
gdf_usig = construye_gdf(fachadas_usig,wkt)

In [ ]:
gdf_sndg = construye_gdf(fachadas_sndg,wkt)

Ahora que finalmente tenemos nuestros geodataframes de puntos listos veamos si las dos herramientas nos devuelven los mismos resultados. Para eso, repasemos algunos conceptos básicos de visualzación. Cuando trabajamos con algunas librerías como matplotlib, es necesario tanto la figura donde se van a plotear nuestros gráficos como el lugar que va a ocupar. En líneas generales, lo que nos debe quedar en claro es que la instancia que nosotros creemos para la figura será siempre el método `figure` del módulo `pyplot` de `matplotlib`. Esto nos permitirá definir atributos como el tamaño. Para la posición de la figura, se utiliza el método `add_subplot` (aplicable siempre a un objeto de tipo `pyplot`. Así, se define la forma en la que se disponen las figuras. En nuestro caso, si queremos que el primer mapa se disponga al lado del segundo o, si por ejemplo, queremos superponerlos. 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# creamos la figura con un tamaño específico
fig = plt.figure(figsize=(15,5))

# disponemos dos ejes en una columna. El tercero que pasamos como parametro es el número de eje
ax1 = fig.add_subplot(1,2,1)
# acá vemos el eje #2
ax2 = fig.add_subplot(1,2,2)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

gdf_usig.plot(ax=ax1, color='red')
gdf_sndg.plot(ax=ax2, color='blue')

ax1.set_axis_off()
ax1.set_title('Resultados Normalizador USIG')
ax2.set_axis_off()
ax2.set_title('Resultados Normalizador SNDG');

In [ ]:
# Si quisieramos verlos superpuestos
fig = plt.figure(figsize=(15,5))
ax = fig.add_subplot(1,1,1)
gdf_usig.plot(ax=ax, color='red')
gdf_sndg.plot(ax=ax, color='blue')
ax.set_axis_off()
ax.set_title('Resultados USIG/SNDG');

Como se puede apreciar, los resultados son bastante similares. Con lo cual, a nivel performance ambos normalizadores trabajaron de manera muy parecida. Pero dónde es que caen esos puntos realmente? En los plots anteriores se puede ver con claridad la forma de la ciudad. De todos modos, es muy común cuando trabajamos con este tipo de situaciones en las que ploteamos algo cuyo resultado inicial nos resulta incierto, querer ver nuestros datos con una especificidad mayor. Hacer zoom, ver en qué polígono cae un punto, el nombre de una calle, etc. Para esto, existen varias librerías que permiten, de una manera muy ágil, plotear figuras sobre un layer dinámico (como si fuese un mapa web). A continuación daremos un pantallazo de algunas de ellas...

> **1. Leaflet**

In [ ]:
# importamos la librería
import mplleaflet

In [ ]:
# Instanciamos el plot
ax = gdf_usig.head(1000).plot(markersize = 50, color = "red")

# Lo visualiamos con el método display
mplleaflet.display(fig=ax.figure)

Si prestamos atención al warning, se nos sugiere utilizar `display` de Ipython. Para ello, vamos a tener que, primero, exportar nuestro mapa y traer la ruta a ese `ħtml` que exportamos. Con leaflet también podemos hacer esto.

In [ ]:
# Ploteamos la data,
gdf_usig.head(1000).plot(markersize = 50, color = "red");
# la exportamos
#mplleaflet.show()

In [ ]:
# y la llamamos nuevamente. IFrame nos permite regular el alto y el ancho del output
path = '_map.html'
from IPython.display import IFrame    
IFrame(path, width=1000, height=400)

> **2. Contextily**

Otra es Contextily, una librería muy útil para plotear puntos, polígonos o líneas sobre un layer estático que nos de un primer pantallazo del mapa que buscamos construir.

In [ ]:
# importamos la librería
import contextily as ctx

In [ ]:
puntos = gdf_usig.plot(figsize=(10, 10), alpha=0.5, color='red', edgecolor='k')
ctx.add_basemap(puntos,zoom=12, crs=4326)
puntos.set_axis_off()

Recuerdan que en la clase anterior, vimos que en la nueva release de `Geopandas` el sistema de proyección de coordenadas (o CRS) había incorporado una mejora en su manera de definirlo? Y recordemos también que, el motivo de este cambio se apoya en la idea de sustitur un `proj4string` (o simplemente un string con información sobre el sistema de proyección utilizado) por un nuevo objeto enriquecido que nos permite acceder a distintos tipos de atributos y métodos. Por ejemplo:

In [ ]:
# esta es la información de nuestro crs
gdf_usig.crs

In [ ]:
# con la que podemos acceder, por ejemplo al nombre
gdf_usig.crs.name

In [ ]:
# o al datum
gdf_usig.crs.datum

Sin embargo, esta nueva forma de definir el objeto CRS puede tener un impacto importante en el uso de distintas librerías. Y seguramente llevará un tiempo hasta que esto termine siendo totalmente compatible o todas se adapten al uso de este nuevo objecto. El principal causal de esto que existen muchas fuentes y formas de definir un CRS, algunas de las cuales pueden tener una descripción que no se ajusta completamente a los nuevos estándares de PROJ> 6 (cadenas de proj4, formatos WKT más antiguos, etc.). Como se menciona en la documentación oficial de geopandas sobre el [uso de proyecciones](https://geopandas.org/projections.html), en estos casos, el objeto pyproj.CRS que se obteiene, podría contener algunas incosistencias. Por ejemplo, un código EPSG que no se condice con la localización esperada.En el [blog](https://jorisvandenbossche.github.io/blog/2020/02/11/geopandas-pyproj-crs/) de [Joris Van der Boschee](https://jorisvandenbossche.github.io/pages/about.html) se brindan algunas explicaciones adicionales bastante interesantes. 

Pero por qué mencionamos esto ahora? Si prestan atención, en la primera instancia del mapa que construimos con `Contextily` el crs que definimos fue el clásico `4326`. ¿Por qué hicimos esto si nosotros construimos el gdf a partir de un `wkt`? Veamos qué hubiese pasado si definíamos el `CRS` de otra manera.

In [ ]:
# Por ejemplo, si hubiésemos querido asignar el crs de otro dataframe cuyo epsg nos es familiar
barrios = gpd.read_file('carto/barrios_badata.shp')
barrios.crs.datum

In [ ]:
# Primero vemos que nos devuelve un error relativo al crs que definimos desde el wkt
puntos = gdf_usig.to_crs(barrios.crs).plot(figsize=(10, 10), alpha=0.5, color='red', edgecolor='k')
ctx.add_basemap(puntos,zoom=12, crs=barrios.crs)

In [ ]:
# Creamos nuestro gdf pero ahora no intentando reproyectar sino asigando el CRS deseado directamente
gdf_usig_crs_barrios = construye_gdf(fachadas_usig,barrios.crs)

In [ ]:
# Y volvemos a intentar la operación
puntos = gdf_usig_crs_barrios.plot(figsize=(10, 10), alpha=0.5, color='red', edgecolor='k')
ctx.add_basemap(puntos,zoom=12, crs=gdf_usig_crs_barrios.crs)

`Contextily` sigue sin reconocer nuestro sistema de coordenadas. Intentemos un camino diferente...

In [ ]:
# Veamos en la descripción de nuestro objeto CRS cual es el código EPSG asignado
gdf_usig_crs_barrios.crs.datum

In [ ]:
# Y intentemos nuevamente...
puntos = gdf_usig_crs_barrios.plot(figsize=(10, 10), alpha=0.5, color='red', edgecolor='k')
ctx.add_basemap(puntos,zoom=12, crs=6221)

Pueden intentarlo tanto con este código como con el que vimos inicialmente en la descripción de nuestro primer CRS (9001). Ambos devolverán un mensaje de error de `CRS no soportado`. Esto, porque `Contextily` no está reconociendo la definición de ninguno de los dos (el que heredamos de nuestro shape de barrios o el especificado en el wkt). 

Y qué hubiese pasado si apelábamos al [EPSG oficial](https://ramsac.ign.gob.ar/posgar07_pg_web/documentos/Informe_sobre_codigos_oficiales_EPSG.pdf) que corresponde a la faja donde cae la Ciudad de Buenos Aires? Pruébenlo y cuéntennos cómo les va!

De manera diferente, esto no nos hubiese pasado, por ejemplo, con `leaflet`. Como dijimos antes, el impacto que tiene este nuevo objeto no es siempre el mismo. Veamoslo:

In [ ]:
# Instanciamos el plot con nuestro nuevo gdf
ax = gdf_usig_crs_barrios.head(1000).plot(markersize = 50, color = "red")

# Y lo visualiamos con el método display
mplleaflet.display(fig=ax.figure)

In [ ]:
# Creamos el gdf con el EPSG que vinos inicialmente en datum
gdf_usig_9001 = construye_gdf(fachadas_usig,9001)

# Y repetimos la operatioria
ax = gdf_usig_9001.head(1000).plot(markersize = 50, color = "red")

mplleaflet.display(fig=ax.figure)

In [ ]:
# Este sería el resultado con Contextily si no se especificara el CRS
puntos = gdf_usig_crs_barrios.plot(figsize=(10, 10), alpha=0.5, color='red', edgecolor='k')
ctx.add_basemap(puntos,zoom=12)

En este tipo de circunstancias, algo que es muy recomendable es trabajar directamente en `4326` (para asegurarnos que no tendremos ningún problema de incompatibilidad o falta de soporte). O bien, este nuevo objecto `pyproj.CRS` cuenta con un método `to_epsg()` para devolver un id equivalente. Este es bastante útil para identificar si nuestra proyección (en nuestro caso definida desde un wkt) está siendo identificada correctamente...

In [ ]:
# En nuestro caso, vemos que el epsg es un None type, algo que ya nos debería parecer sospechoso
print(gdf_usig.crs.to_epsg())

In [ ]:
# También podemos acceder a la descripción original si hacemos:
gdf_usig.crs.source_crs

In [ ]:
# Y veamos también que ahora trabajamos con un nuevo parámetro: min_confidence
gdf_usig.crs.source_crs.to_epsg(min_confidence=20)

Esto se encuentra bastante bien explicado en la documentación de [pyproj](https://pyproj4.github.io/pyproj/stable/gotchas.html#what-is-the-best-format-to-store-the-crs-information). Si el método `to_epsg` no nos devuelve ningún código EPSG que coincida con el wkt o el pyproj4 string que usamos inicialmente (porque no lo encuentra o porque no existe), podemos apelar al parámetro `min_confidence`. Este, nos permitiría obtener el código EPSG más cercano, alternando los umbrales que estamos dispuestos a tolerar.

In [ ]:
# Vemos que ahora Contextily interpeta el crs...
puntos = gdf_usig.plot(figsize=(10, 10), alpha=0.5, color='red', edgecolor='k')
ctx.add_basemap(puntos,zoom=12, crs=gdf_usig.crs.source_crs.to_epsg(min_confidence=20))

En este caso, si bien el resultado es bastante similar al que obtuvimos con un EPSG igual a 4326, este método nos puede resultar de utilidad para situaciones en las que necesitemos trabajar con una proyección diferente asegurándonos que el mismo sea interpretable.

> **3. Folium**

Folium es una librería que combina distintos tipos de funcionalidades. Desde controles de zoom y la posibilidad de alternar layers hasta clases que devuelven mapas de calor y coropletas.

In [ ]:
# Si no lo incluiste en los requirements, lo podemos instalar directo desde el notebook
#!pip install folium

In [ ]:
# Importamos la librería
import folium

In [ ]:
# Creamos un nuevo gdf en 4326 para los resultados de ambos nomralizadores
sndg_4326 = construye_gdf(fachadas_sndg,4326)

In [ ]:
usig_4326 = construye_gdf(fachadas_usig,4326)

In [ ]:
# Convertimos nuestro gdf de puntos en geojson
sndg_gjson = folium.features.GeoJson(sndg_4326.head(1000), name="SNDG")

# Acá podemos ver la estructura
#points_gjson.data.get('features')

# Y ploteamos el mapa...
mapa = folium.Map(location=[-34.6157437, -58.4333812], 
               zoom_start=11, 
               control_scale=True)
sndg_gjson.add_to(mapa);

In [ ]:
mapa

In [ ]:
def mapa_folium(gdf, polygons, barrio, exporta):
    '''
    Construye un mapa de puntos y/o polígonos sobre
    un layer dinámico de leaflet.
    ...
    Argumentos:
     gdf(GeoDataFrame): Geodataframe de puntos.
     barrio (str): nombre del barrio.
     polygons (GeoDataFrame): GeoDataFrame de polígonos.
     exporta (bool): True para exportar.
     
    Devuelve:
      folium.Map : mapa de puntos/polígonos  
    '''
    
    # Creamos el layer de leaflet. Lo centramos en la Ciudad de Buenos Aires
    centroide =  gdf.geometry.centroid
    coordenadas = [centroide.y.mean(), centroide.x.mean()]
    layer = folium.Map(location=coordenadas, 
                       zoom_start=11, 
                       height = 500,
                       control_scale=True)
    
    
    # Si pasamos polígonos de base, también los cargamos al mapa
    if polygons is not None:
        estilo = lambda x: {'fillColor': 'red' if
                            x['properties']['BARRIO']== barrio else
                            'grey'}
        
        pol_hover= folium.features.GeoJsonTooltip(
                        fields=['BARRIO','COMUNA'],
                        aliases=['Barrio:','Comuna:'])
        
        folium.GeoJson(polygons.to_crs(gdf.crs),
                       name = 'Barrios de la ciudad',
                       style_function = estilo,
                       tooltip = pol_hover
                       ).add_to(layer)

    
    # Agregamos nuestros puntos con el método CircleMarker
    for i, row in gdf.iterrows():
        folium.CircleMarker((row.lat,row.lon),            
                            radius=3, weight=2, 
                            color='blue', 
                            fill_color='blue', 
                            fill_opacity=.5,
                            tooltip= 'Certificado:'+row.vencimiento_certificado).add_to(layer)
    
    
    # Exportar el resultado
    if exporta:
        layer.save('mapa.html')
    
    folium.LayerControl(autoZIndex=False, collapsed=False).add_to(layer)
    
    return layer

In [ ]:
# ploteamos un mapa sólo de puntos
mapa_folium(sndg_4326.head(50), None, None, False)

In [ ]:
# ahora lo combinamos con un gdf de polígonos
mapa_folium(sndg_4326.head(1000), barrios, 'VILLA LUGANO', False)

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

barrios.plot(ax=ax1, color='grey')
usig_4326.to_crs(barrios.crs).plot(ax=ax1, color='red', markersize=1)

barrios.plot(ax=ax2, color='grey')
sndg_4326.to_crs(barrios.crs).plot(ax=ax2, color='blue', markersize=1)


ax1.set_axis_off()
ax2.set_axis_off()

ax1.set_title('Resultados Normalizador USIG')
ax2.set_title('Resultados Normalizador SNDG');

In [ ]:
fig = plt.figure(figsize=(15,5))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

barrios.plot(ax=ax1, color='grey')
usig_4326.to_crs(barrios.crs).plot(ax=ax1, color='red', markersize=10, edgecolor='white')

barrios.plot(ax=ax2, color='grey')
sndg_4326.to_crs(barrios.crs).plot(ax=ax2, color='blue', markersize=10, edgecolor='white')


ax1.set_axis_off()
ax2.set_axis_off()

ax1.set_title('Resultados Normalizador USIG')
ax2.set_title('Resultados Normalizador SNDG');

In [ ]:
from folium.plugins import HeatMap

In [ ]:
def mapa_de_calor(gdf, radio, inicio, fin):
    '''
    Construye un mapa de calor sobre
    un layer dinámico de leaflet.
    ...
    Argumentos:
     gdf(GeoDataFrame): Geodataframe de puntos.
     radio (int): radio de cada punto.
     inicio (int): límite inferior.
     fin (int): límite superior.
     
    Devuelve:
      folium.Map : mapa de calor  
    '''
    
    # Ubicamos el centro del mapa
    centroide =  gdf.geometry.centroid
    coordenadas = [centroide.y.mean(), centroide.x.mean()]
    
    # Conseguimos la latitud y la longitud de cada punto con una función anónima
    x = gdf["geometry"].apply(lambda punto: punto.x)
    y = gdf["geometry"].apply(lambda punto: punto.y)

    # Convertimos esas coordenadas en una lista de tuplas
    puntos = list(zip(y, x))
    
    # Creamos el layer de leaflet
    layer = folium.Map(location=coordenadas, 
                       zoom_start=11, control_scale=True)

    # Agregamos el mapa de calor al layer que habíamos instanciado
    HeatMap(puntos[inicio:fin], 
            radius=radio).add_to(layer)

    return layer

In [ ]:
# también se puede jugar con otros parametros como max_zoom o el tile para elegir otro base map
mapa_de_calor(usig_4326, 15, 1000, 2000)

Para concluir con esta práctica, veamos qué se puede hacer con los datasets que hemos venido construyendo. Un aspecto muy importante en este tipo de situaciones en las que la construcción de nuestros set de datos se vuelve un proceso lento y demandante, es no olvidar o dejar de lado la pregunta que nos va a ayudar a responder. Y no por la pregunta en sí, ya que puede ser una o más. Sino porque, la comunicación de nuestros hallazgos se vuelve mucho más clara cuando respondemos algo en concreto. Por ejemplo, dónde se aglomera nuestra variable de interés? existe algún patrón de localización específico?

Con nuestro mapa de calor, ya pudimos identificar algunas posibles zonas. Sin embargo, vamos a aprovechar esta pregunta para introducir una nueva geometría, un tipo de polígono que no vimos hasta ahora. Este no es nada más y nada menos que el radio censal. Una división o unidad geográfica que utiliza el [Instituto Nacional de Estadísticas y Censos (INDEC)](https://www.indec.gob.ar/indec/web/Institucional-Indec-Codgeo) para sus relevamientos censales. 

Pero no vamos a ahondar mucho en este tema. Simplemente, mencionaremos que una de sus principales ventajas, es la posibilidad de contar con información sociodemográfica a un alto nivel de desagregación territorial (algo que se vuelve importante en términos de acceso a información útil para describir con detalle áreas más reducidas - o con mayor zoom). Tema que retomaremos en otro notebook.

A continuación, veremos un ejemplo en el que utilizaremos los radios solamente por el tamaño de los polígonos (también existen otras librerías muy útiles para lograr este tipo de efectos en nuestros outputs). Haremos un join espacial para ver la cantidad de puntos (nuestras fachadas registradas en las partidas de conservación patrimonial) que caen dentro de nuestros polígonos censales y evaluaremos la concentración de actividad (personas que hayan registrado una fachada) en base a la clasificación de este valor dentro de los radios.

Como vimos en la clase anterior, `Geopandas` cuenta con un módulo bastante útil que nos permite detectar la relación espacial entre dos geometrías. Este se denomina [spatial join](https://geopandas.org/mergingdata.html#spatial-joins) y nos permitira conocer cuántos puntos caen dentro del área de cada polígono.

In [ ]:
# importamos el módulo desde gepandas
from geopandas import sjoin

In [ ]:
# cargamos el shapefile de radios censales
radios = gpd.read_file('carto/informacion_censal_por_radio_2010.shp')

In [ ]:
# mergeamos ambos gdf
fachadas_r = gpd.sjoin(sndg_4326, radios.to_crs(sndg_4326.crs))

In [ ]:
# visualizamos el reultado...
fachadas_r.plot();

In [ ]:
# Agrupamos la cantidad de fachadas con certificado de conservación por radio censal
fachadas_por_radio = fachadas_r.groupby(['RADIO_I'])[['partida']].count().reset_index()

In [ ]:
# Creamos un diccioanrio donde la key es el id del radio y el value la cantidad de partidas.
d = dict(zip(fachadas_por_radio['RADIO_I'], fachadas_por_radio['partida']))

In [ ]:
# Agregamos esta nueva información como columna adicional de nuestro shape original de radios.
radios['partidas'] = radios['RADIO_I'].map(d)

In [ ]:
# Hacemos una primera visualización de este resultado.
fig = plt.figure(figsize=(15,7))
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

radios.to_crs(wkt).plot(ax=ax1, column='partidas', scheme='Natural_Breaks', k=3, 
                        linewidth=0.2, edgecolor='black', cmap='Reds_r', legend=True)
radios.to_crs(wkt).plot(ax=ax2, column='partidas', scheme='Quantiles', k=3, 
                        linewidth=0.2, edgecolor='black', cmap='Blues_r', legend=True)

ax1.set_axis_off()
ax2.set_axis_off()

Vamos a introducir otro tema que retomaremos en el notebook siguiente: esquemas de clasificación. Acá simplemente vamos a comparar dos distintos para detectar si existe algún tipo de diferencia. Como no se pueden apreciar contrastes importantes sólo nos detendremos brevemente en la última distribución. Los `quantiles`.

In [ ]:
# reemplazamos los casos sin valor por 0 (en nuestro join espacial muchos polígonos no coincidían con ningún punto)
radios['partidas'].fillna(0,inplace=True)

In [ ]:
# calculamos nuestros bins de corte (quantiles). 
qcut = list(radios['partidas'].quantile([0, 0.25, 0.5, 0.75, 1]))

In [ ]:
# Esto nos devuelve cinco intervalos, o quintiles que dividen a nuestra población en cinco partes iguales
pd.qcut(radios['partidas'], 5, labels=False).value_counts()

Qué significa esto? Básicamente que el primer 20% de nuestros radios censales no tendrá fachadas o partidas dentro del área delimitada por los bordes del polígono. El segundo 20%, es decir entre el 20 y el 40% de los casos tendrá un máximo de 2 partidas. Del 40 al 60 será entre 2 y 5, del 60 al 80% entre 5 y 10 y el último quintil (o 20% de nuestros casos) trendrá entre 10 y 75 partidas. Es decir, el más disperso. Pero como dijimos, este tema lo retomaremos en nuestra próxima clase cuando revisemos algunas clases y métodos de la librería `mapclassify`.

In [ ]:
# nuestros bins de corte
qcut

Ahora, vamos a utilizar la clase `Choropleth` de `Folium` para plotear nuestro agrupamiento por radio. Presten atención a la lógica general. Esta no difiere de los otros mapas que hemos visto. Primero agregar el mapa con `.Map()`(para lo que dejamos seteado un camino que siempre nos devuelva el centro de nuestro gdf a partir de los ceontroides de las latitudes y longitudes). Y luego, ir creando los distintos objetos que iremos agregando como capas superpuestas.

In [ ]:
# creamos el mapa
gdf = radios.copy()
centroide =  gdf.geometry.centroid
coordenadas = [centroide.y.mean(), centroide.x.mean()]
layer = folium.Map(location=coordenadas, 
                   zoom_start=11, 
                   height = 500,
                   control_scale=True)

# esto nos permitiría seleccionar entre distintos tiles de base
tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']
for tile in tiles:
    folium.TileLayer(tile).add_to(layer)

# instanciamos la coropleta
fachadas = folium.Choropleth(name='Radios censales',
                             geo_data=gdf,
                             data=gdf[[c for c in gdf.columns if c!='geometry']],
                             columns=['RADIO_I','partidas'],
                             key_on='properties.RADIO_I',
                             fill_color='Reds_r', 
                             bins = qcut,
                             fill_opacity=0.6, 
                             line_opacity=0.2,
                             highlight=True,
                             legend_name='Cantidad de partidas por radio censal (CNPHV-2010)',
                             smooth_factor=1).add_to(layer)

# creamos un tooltip para hacer un hover en nuestros polígonos
pol_hover = folium.features.GeoJsonTooltip(fields=['BARRIO','RADIO_I','partidas'],
                                           aliases=['Barrio:','Radio_id:','Partidas:'])

# Y lo agregamos al layer inicial
folium.GeoJson(gdf,
               style_function=lambda x: {"weight":0.35, 
                                         'color':'white', 
                                         'fillOpacity':0.0},
               smooth_factor=2.0,
               tooltip=pol_hover).add_to(fachadas);

In [ ]:
layer

Es importante remarcar que, en este ejemplo, hemos pasado un geodataframe a la clase `Choropleth` de folium. También podríamos haberle pasado directamente un objeto de tipo `GeoJson`, que es lo que la clase usa para mapear nuestros polígonos (buscar la key, aplicar estilos, etc.). De hecho, si se fijan en el parámetro `key_on`, se accede a la columna de radios a través de la key `properties`. Veamos cómo es eso:

In [ ]:
#import json

In [ ]:
# cargamos un geojson de radios censales (es nuestro mismo gdf que previamente exportamos en este formato)
with open('radios.geojson') as f:
    rg = json.load(f)

In [ ]:
# Vemos las keys...
rg.keys()

In [ ]:
# y adentro de features, nuestras columnas!
rg['features']

Este primer resultado nos permite ver dos cosas. Primero, una división fuerte entre dos grupos. Si bien nuestro último quintil tiene un amplio rango entre la base y el techo del intervalo, nos sirve para diferenciar dos áreas de la Ciudad. aquellas con nula o casi nula actividad y aquellas donde los radios tienen un número de certificados de conservación superior al techo del resto. 

En otras palabras, los radios censales con más de diez fachadas registradas se concentran desde el centro hacia el oeste, siguiendo la traza de la Avenida Rivadavia hasta poco más de Primera Junta. Y del centro hacia el Norte. Aprovechemos para agregar alguna traza que nos permita ver este efecto con mayor claridad.

In [ ]:
# Instanciamos el callejero de la Ciudad, que también bajamos de Buenos Aires Data.
callejero = gpd.read_file('carto/callejero.shp')

In [ ]:
# Filtramos las avenidas
avenidas = callejero.loc[callejero['tipo_c']=='AVENIDA']

In [ ]:
# Nos quedamos con algunas trazas que vayan del centro al oeste y del centro al norte
ppales = avenidas.loc[(avenidas['nom_mapa']=='AV. DE MAYO')|(avenidas['nom_mapa']=='AV. RIVADAVIA')|
                     (avenidas['nom_mapa']=='AV. CORRIENTES')|(avenidas['nom_mapa']=='AV. SANTA FE')|
                     (avenidas['nom_mapa']=='AV.CABILDO')]

In [ ]:
# Cortamos rivadavia y eliminamos algunas secciones con altura cero
pp_clean = ppales.loc[(ppales['nom_mapa']=='AV. DE MAYO')|
                      ((ppales['nom_mapa']=='AV. RIVADAVIA')&
                      (ppales['alt_izqfin']<8000)&(ppales['alt_derfin']<8000)&
                      # tramos de rivadavia con altura = 0
                      (ppales['id']!=19894)&(ppales['id']!=19844)&(ppales['id']!=19104)&(ppales['id']!=31505))|
                      (ppales['nom_mapa']=='AV. CORRIENTES')|(ppales['nom_mapa']=='AV. SANTA FE')|
                      (ppales['nom_mapa']=='AV.CABILDO')]

In [ ]:
# Volvemos a plotear nuestra coropleta. 
gdf = radios.copy()
centroide =  gdf.geometry.centroid
coordenadas = [centroide.y.mean(), centroide.x.mean()]
layer = folium.Map(location=coordenadas, 
                   zoom_start=11, 
                   height = 500,
                   control_scale=True)

tiles = ['stamenwatercolor', 'cartodbpositron', 'openstreetmap', 'stamenterrain']
for tile in tiles:
    folium.TileLayer(tile).add_to(layer)


fachadas = folium.Choropleth(name='Radios censales',
                             geo_data=gdf,
                             data=gdf[[c for c in gdf.columns if c!='geometry']],
                             columns=['RADIO_I','partidas'],
                             key_on='properties.RADIO_I',
                             fill_color='Reds_r', 
                             bins = qcut,
                             fill_opacity=0.6, 
                             line_opacity=0.2,
                             highlight=True,
                             legend_name='Cantidad de partidas por radio censal (CNPHV-2010)',
                             smooth_factor=1).add_to(layer)

pol_hover = folium.features.GeoJsonTooltip(fields=['BARRIO','RADIO_I','partidas'],
                                           aliases=['Barrio:','Radio_id:','Partidas:'])

folium.GeoJson(gdf,
               style_function=lambda x: {"weight":0.35, 
                                         'color':'white', 
                                         'fillOpacity':0.0},
               smooth_factor=2.0,
               tooltip=pol_hover).add_to(fachadas)


# ...pero ahora agregando un layer de calles.
folium.GeoJson(pp_clean,
               name='Avenidas',
               style_function=lambda x: {'weight':2,'color':'#355C7D'},
               highlight_function=lambda x: {'weight':5,'color':'yellow'},
               tooltip=folium.features.GeoJsonTooltip(fields=['nom_mapa','alt_izqfin','alt_derfin'],
                                                      aliases=['Calle:','Altura(izq):','Altura(der):']),
               smooth_factor=5.0,
              ).add_to(layer)

# También agregamos un selector de layers
folium.LayerControl().add_to(layer);

In [ ]:
layer

Ahora sí, nuestra choropleta está lista. Qué podemos ver? Inicialmente que la concentración más alta queda denotada por el mismo modelo territorial de la ciudad. Del centro al oeste y del centro al norte (lo que se puede ver a través de las trazas de las avenidas que seleccionamos). Como dijimos anteriormente, si bien el rango es alto (de 10 a 75), nos permite ver una clara diferencia con el resto de la ciudad. Allí es donde se concentran las fachadas que han sido patrimoniadas. Un segundo paso podría ser filtrar este universo y evaluar dónde es que vuelven a concentrarse. Pero ese análisis se lo dejamos a ustedes!